## MS Example 2: Additive Noise, Surrogate-Based Methods

In [1]:
import numpy as np
import dfo_v1 as dfo
import ECNoise as ECN
import active_subspaces as ac
import matplotlib.pyplot as plt

We let $f: \mathbf{\lambda}: \mathbb{R}^{11} \to \mathbb{R}$, where $$f(\lambda; \xi)=\sum_{i=0}^{10} 2^{(-1)^i i}\lambda_i^2+\epsilon(\xi), \quad \epsilon(\xi) \sim U[-k,k], \quad k=1 \times 10^{-2}.$$

In [2]:
k=1E-2

dim=11

weights=np.zeros((1,dim)) # row vec to make IP easy
for i in range(dim):
    weights[:,i]=2**(((-1)**i)*i)

def tfun(x,var=k):
    y=np.copy(x)
    y=y**2   
    return np.dot(weights,y)*(1+k*(2*np.random.rand()-1))

## Sample 100 random draws in $\Lambda$, build a surrogate, minimize

In [3]:
# Perform draws
n_samps=100

X=np.random.rand(dim,n_samps)
samps=2*X-1
print(samps.shape,samps[:,0:1])

(11, 100) [[-0.1494416 ]
 [ 0.8751241 ]
 [ 0.26666952]
 [-0.66734957]
 [ 0.40247732]
 [-0.76564116]
 [ 0.46512422]
 [ 0.71282808]
 [ 0.99570347]
 [-0.36469183]
 [-0.43363187]]


In [4]:
# Compute the f values

f_vals=np.zeros((n_samps,1))

for i in range(n_samps):
    
    f_vals[i] = tfun(samps[:,i:i+1])
    
print(f_vals.shape)

(100, 1)


In [5]:
ss = ac.subspaces.Subspaces()

trainx=np.transpose(samps)

# Use Global Quadratic
ss.compute(X=trainx, f=f_vals, nboot=0, sstype='QPHD')
    
RS=ac.utils.response_surfaces.PolynomialApproximation(2)
    
y1 = trainx.dot(ss.W1)
    
RS.train(y1, f_vals)

print(ss.W1)

print()

print(np.sqrt(ss.eigenvals))

n should be an integer. Performing conversion.
[[ 7.00478262e-04]
 [ 2.27711323e-03]
 [ 9.74564790e-04]
 [ 1.43688181e-03]
 [ 4.23793874e-04]
 [ 1.76129437e-03]
 [-5.22044096e-05]
 [ 7.63276183e-04]
 [-6.07692763e-04]
 [-2.27750589e-03]
 [-9.99990943e-01]]

[[1.18231831e+03]
 [3.00080062e+02]
 [7.24879718e+01]
 [1.88930948e+01]
 [8.82636002e+00]
 [6.91380080e+00]
 [3.98422309e+00]
 [3.33908743e+00]
 [2.68841720e+00]
 [1.15823818e+00]
 [3.53753941e-01]]


In [6]:
avdom=ac.domains.BoundedActiveVariableDomain(ss)
    
ystar, fstar = ac.optimizers.av_minimize(lambda x: RS.predict(x.reshape(1,1))[0], avdom)

print(ystar,fstar,RS.Rsqr)

[[-0.00269124]] [[108.96114737]] 0.9369650194649976
